<a href="https://colab.research.google.com/github/lorenzinigiovanni/taaocv-project/blob/main/fsgan_face_swapping.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## FSGAN Face Swapping Demo
Fill out [this form](https://docs.google.com/forms/d/e/1FAIpQLScyyNWoFvyaxxfyaPLnCIAxXgdxLEMwR9Sayjh3JpWseuYlOA/viewform?usp=sf_link),
and after receiving the email, add the FSGAN shared directory to your drive:

![Add to drive](https://raw.githubusercontent.com/wiki/YuvalNirkin/fsgan/media/add_to_drive.jpg)




Acknowledgements: We thank Dr. Eyal Gruss, [wangchao0899](https://github.com/wangchao0899), [jjandnn](https://github.com/jjandnn), and [zhuhaozh](https://github.com/zhuhaozh) for helping with this demo.

A Tesla P100 GPU is recommended for this demo. Let's see what we got:

In [ ]:
!nvidia-smi

### Installation

In [ ]:
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local
!rm Miniconda3-latest-Linux-x86_64.sh
!mkdir -p /content/projects/
%cd /content/projects/
!git clone https://github.com/YuvalNirkin/fsgan
%cd fsgan
!conda install pytorch=1.12.1 \
    torchvision=0.13.1 \
    cudatoolkit=11.6 \
    pip=20.3.1 \
    ffmpeg=4.4.1 \
    yacs=0.1.8 \
    -c pytorch \
    -c conda-forge \
    -c anaconda \
    -y

!pip install setuptools==58.2.0 \
    torch-summary==1.4.5 \
    opencv-contrib-python==4.5.4.60 \
    tensorflow==2.7.0 \
    tqdm==4.64.1 \
    matplotlib==3.6.2 \
    ffmpeg-python==0.2.0 \
    PyYAML==6.0 \
    pandas==1.5.1 \
    seaborn==0.12.1 \
    scipy==1.9.3 \
    ipython==7.9.0 \
    youtube-dl \
    git+https://github.com/YuvalNirkin/face_detection_dsfd.git

!pip install /content/projects/fsgan

!mkdir -p /content/data
!cp /content/projects/fsgan/docs/examples/shinzo_abe.mp4 /content/data/source.mp4
!cp /content/projects/fsgan/docs/examples/conan_obrien.mp4 /content/data/target.mp4

import sys
sys.path += ['/usr/local/lib/python3.9/site-packages']
    

Mount your Google Drive using the following step or click on "Mount Drive" in the menu to the left

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Initialize useful functions for later

In [4]:
# Utility functions
import ffmpeg
from base64 import b64encode
from IPython.display import HTML, clear_output

def encode_audio(video_path, audio_path, output_path):
  ffmpeg.concat(ffmpeg.input(video_path), ffmpeg.input(audio_path), v=1, a=1) \
    .output(output_path, strict='-2').run(overwrite_output=True)


def display_video(video_path, width=640, clear=True):
  vid_data = open(video_path,'rb').read()
  vid_url = 'data:video/mp4;base64,' + b64encode(vid_data).decode()

  if clear:
    clear_output()

  return HTML(f"""
  <video width={width} controls>
    <source src={vid_url} type="video/mp4">
  </video>
  """)

### Apply face swapping
Remember to modify the parameters on the right as you wish.

In [7]:
import time, os, fnmatch, shutil
from fsgan.inference.swap import FaceSwapping
from fsgan.criterions.vgg_loss import VGGLoss

#@markdown This step should only be done once unless one of the
#@markdown following parameters is changed:

#@markdown ---
#@markdown Path to the weights directory (make sure it is correct):
weights_dir = '/content/drive/My Drive/FSGAN/fsgan/weights' #@param {type:"string"}
#@markdown Number of finetune iterations on the source subject:
finetune_iterations = 800 #@param {type:"slider", min:100, max:2000, step:1}
#@markdown If True, the inner part of the mouth will be removed from the segmentation:
seg_remove_mouth = True #@param {type:"boolean"}
#@markdown Segmentation batch size
seg_batch_size = 24 #@param {type:"slider", min:1, max:64, step:1}
#@markdown Inference batch size
batch_size = 8 #@param {type:"slider", min:1, max:64, step:1}
#@markdown ---


detection_model = os.path.join(weights_dir, 'v2/WIDERFace_DSFD_RES152.pth')
pose_model = os.path.join(weights_dir, 'shared/hopenet_robust_alpha1.pth')
lms_model = os.path.join(weights_dir, 'v2/hr18_wflw_landmarks.pth')
seg_model = os.path.join(weights_dir, 'v2/celeba_unet_256_1_2_segmentation_v2.pth')
reenactment_model = os.path.join(weights_dir, 'v2/nfv_msrunet_256_1_2_reenactment_v2.1.pth')
completion_model = os.path.join(weights_dir, 'v2/ijbc_msrunet_256_1_2_inpainting_v2.pth')
blending_model = os.path.join(weights_dir, 'v2/ijbc_msrunet_256_1_2_blending_v2.pth')
criterion_id_path = os.path.join(weights_dir, 'v2/vggface2_vgg19_256_1_2_id.pth')
criterion_id = VGGLoss(criterion_id_path)


face_swapping = FaceSwapping(
    detection_model=detection_model, pose_model=pose_model, lms_model=lms_model,
    seg_model=seg_model, reenactment_model=reenactment_model,
    completion_model=completion_model, blending_model=blending_model,
    criterion_id=criterion_id,
    finetune=True, finetune_save=True, finetune_iterations=finetune_iterations,
    seg_remove_mouth=finetune_iterations, batch_size=batch_size,
    seg_batch_size=seg_batch_size, encoder_codec='mp4v')
  
# Do face swapping
#@markdown ---
#@markdown Toggle whether to finetune the reenactment generator:
finetune = True #@param {type:"boolean"}
#@markdown Source path
source_path = '/content/drive/My Drive/FSGAN/data/source.mp4' #@param {type:"string"}
#@markdown Source selection method ["longest" | sequence number]:
select_source = 'longest' #@param {type:"string"}
#@markdown Target path
target_path = '/content/drive/My Drive/FSGAN/data/target.mp4' #@param {type:"string"}
#@markdown Target selection method ["longest" | sequence number]:
select_target = 'longest' #@param {type:"string"}
#@markdown ---
output_tmp_path = '/content/data/output_tmp.mp4'

t = time.localtime()
timestamp = time.strftime('%b-%d-%Y_%H%M', t)

output_path = '/content/drive/My Drive/FSGAN/output/output-' + timestamp + '.mp4'
face_swapping(source_path, target_path, output_tmp_path,
              select_source, select_target, finetune)

# Encode with audio and display result
encode_audio(output_tmp_path, target_path, output_path)
os.remove(output_tmp_path)
display_video(output_path)